In [1]:
from __future__ import absolute_import, division, print_function
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [2]:
#Importing imdb library from tensorflow
imdb = keras.datasets.imdb
#Loading data from imdb library to respective variables
(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=10000)

In [3]:
#Printing Size of Training DataSets
print("Training entries: {}, labels: {}".format(len(train_data), len(train_labels)))

Training entries: 25000, labels: 25000


In [4]:
word_index = imdb.get_word_index()
word_index = {k:(v+3) for k,v in word_index.items()} 
word_index["<PAD>"] = 0 #To indicate padding in a review
word_index["<START>"] = 1 #To indicate start of a review
word_index["<UNK>"] = 2  #To indicate if a word isn't present in the top 10000 frequent words
word_index["<UNUSED>"] = 3

In [5]:
#Making all sentences(reviews) in training dataset of uniform length (256 words)
train_data = keras.preprocessing.sequence.pad_sequences(train_data,
                                                        value=word_index["<PAD>"],
                                                        padding='post',
                                                        maxlen=256)
#Making all sentences(reviews) in testing dataset of uniform length (256 words)
test_data = keras.preprocessing.sequence.pad_sequences(test_data,
                                                       value=word_index["<PAD>"],
                                                       padding='post',
                                                       maxlen=256)

In [6]:
vocab_size = 10000  #Number of most frequent words kept in the embedding vector matrix

In [10]:
model = keras.Sequential()
model.add(keras.layers.Embedding(vocab_size,100,input_length=256)) #Converting word to equivalent vector while improving vector while still training per epoch
model.add(keras.layers.Conv1D(filters=100,kernel_size=5,strides=3,activation='relu')) #Convolutional Layer 1
model.add(keras.layers.Dropout(rate=0.5)) #Dropout Layer 1 (To avoid overfitting)
model.add(tf.keras.layers.MaxPooling1D(pool_size=2,strides=2)) #Pooling Layer 1 (To take out the important data from Convolutional layer 1)
model.add(keras.layers.Conv1D(filters=100,kernel_size=5,strides=3,activation='relu')) #Convolutional Layer 2
model.add(keras.layers.Dropout(rate=0.5)) #Dropout Layer 2 (To avoid overfitting)
model.add(tf.keras.layers.MaxPooling1D(pool_size=2,strides=2)) #Pooling Layer 2 (To take out the important data from Convolutional layer 2)
model.add(keras.layers.Conv1D(filters=100,kernel_size=1,strides=3,activation='relu')) #Convolutional Layer 3
model.add(keras.layers.Dropout(rate=0.5)) #Dropout Layer 3 (To avoid overfitting)
model.add(tf.keras.layers.MaxPooling1D(pool_size=1,strides=1)) #Pooling Layer 3 (To take out the important data from Convolutional layer 3)
model.add(keras.layers.Flatten()) #Converting to single array of vectors
model.add(keras.layers.Dense(100)) #Feeding to dense layer with 100 neurons
model.add(keras.layers.Dense(1, activation=tf.nn.sigmoid)) #Feeding to dense layer with 1 neuron
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 256, 100)          1000000   
_________________________________________________________________
conv1d_6 (Conv1D)            (None, 84, 100)           50100     
_________________________________________________________________
dropout_6 (Dropout)          (None, 84, 100)           0         
_________________________________________________________________
max_pooling1d_6 (MaxPooling1 (None, 42, 100)           0         
_________________________________________________________________
conv1d_7 (Conv1D)            (None, 13, 100)           50100     
_________________________________________________________________
dropout_7 (Dropout)          (None, 13, 100)           0         
_________________________________________________________________
max_pooling1d_7 (MaxPooling1 (None, 6, 100)            0         
__________

In [11]:
sgd=keras.optimizers.SGD(lr=0.001, momentum=0.9, decay=0.0005, nesterov=True) #Using Scholastic Gradient Descent as optimizer

In [12]:
model.compile(optimizer = sgd,loss='binary_crossentropy',metrics=['acc']) #Compiling the model

In [13]:
validdata = train_data[:10000] #Validation Data
traindata = train_data[10000:] #Training Data

validlabels = train_labels[:10000] #Validation Labels (Sentiment)
trainlabels = train_labels[10000:] #Training Labels (Sentiment)

In [42]:
#Training model (Total of 7000 epochs)
history = model.fit(traindata,
                    trainlabels,
                    epochs=500,
                    batch_size=100,
                    validation_data=(validdata, validlabels),
                    verbose=1,
                   shuffle=True)

Train on 15000 samples, validate on 10000 samples
Epoch 1/500
15000/15000 [==============================] - 8s 566us/sample - loss: 0.4265 - acc: 0.8067 - val_loss: 0.4770 - val_acc: 0.7982
Epoch 2/500
15000/15000 [==============================] - 8s 560us/sample - loss: 0.4247 - acc: 0.8111 - val_loss: 0.4770 - val_acc: 0.7981
Epoch 3/500
15000/15000 [==============================] - 8s 557us/sample - loss: 0.4288 - acc: 0.8033 - val_loss: 0.4770 - val_acc: 0.7983
Epoch 4/500
15000/15000 [==============================] - 8s 561us/sample - loss: 0.4306 - acc: 0.8051 - val_loss: 0.4770 - val_acc: 0.7978
Epoch 5/500
15000/15000 [==============================] - 8s 562us/sample - loss: 0.4273 - acc: 0.8039 - val_loss: 0.4770 - val_acc: 0.7983
Epoch 6/500
15000/15000 [==============================] - 9s 567us/sample - loss: 0.4236 - acc: 0.8101 - val_loss: 0.4770 - val_acc: 0.7984
Epoch 7/500
15000/15000 [==============================] - 9s 586us/sample - loss: 0.4270 - acc: 0.8079 

15000/15000 [==============================] - 10s 688us/sample - loss: 0.4221 - acc: 0.8098 - val_loss: 0.4758 - val_acc: 0.7986
Epoch 116/500
15000/15000 [==============================] - 10s 693us/sample - loss: 0.4267 - acc: 0.8086 - val_loss: 0.4758 - val_acc: 0.7986
Epoch 117/500
15000/15000 [==============================] - 10s 688us/sample - loss: 0.4248 - acc: 0.8090 - val_loss: 0.4758 - val_acc: 0.7987
Epoch 118/500
15000/15000 [==============================] - 10s 687us/sample - loss: 0.4218 - acc: 0.8106 - val_loss: 0.4758 - val_acc: 0.7986
Epoch 119/500
15000/15000 [==============================] - 10s 697us/sample - loss: 0.4248 - acc: 0.8119 - val_loss: 0.4757 - val_acc: 0.7986
Epoch 120/500
15000/15000 [==============================] - 10s 694us/sample - loss: 0.4219 - acc: 0.8116 - val_loss: 0.4757 - val_acc: 0.7986
Epoch 121/500
15000/15000 [==============================] - 10s 687us/sample - loss: 0.4215 - acc: 0.8134 - val_loss: 0.4757 - val_acc: 0.7988
Epoch 

Epoch 172/500
15000/15000 [==============================] - 10s 679us/sample - loss: 0.4250 - acc: 0.8077 - val_loss: 0.4751 - val_acc: 0.7986
Epoch 173/500
15000/15000 [==============================] - 10s 681us/sample - loss: 0.4236 - acc: 0.8116 - val_loss: 0.4751 - val_acc: 0.7991
Epoch 174/500
15000/15000 [==============================] - 10s 678us/sample - loss: 0.4270 - acc: 0.8053 - val_loss: 0.4751 - val_acc: 0.7990
Epoch 175/500
15000/15000 [==============================] - 10s 677us/sample - loss: 0.4259 - acc: 0.8078 - val_loss: 0.4751 - val_acc: 0.7990
Epoch 176/500
15000/15000 [==============================] - 10s 678us/sample - loss: 0.4253 - acc: 0.8059 - val_loss: 0.4751 - val_acc: 0.7991
Epoch 177/500
15000/15000 [==============================] - 10s 685us/sample - loss: 0.4218 - acc: 0.8080 - val_loss: 0.4751 - val_acc: 0.7989
Epoch 178/500
15000/15000 [==============================] - 10s 684us/sample - loss: 0.4278 - acc: 0.8077 - val_loss: 0.4750 - val_acc:

Epoch 229/500
15000/15000 [==============================] - 10s 674us/sample - loss: 0.4243 - acc: 0.8080 - val_loss: 0.4745 - val_acc: 0.7989
Epoch 230/500
15000/15000 [==============================] - 10s 674us/sample - loss: 0.4262 - acc: 0.8071 - val_loss: 0.4745 - val_acc: 0.7989
Epoch 231/500
15000/15000 [==============================] - 10s 685us/sample - loss: 0.4259 - acc: 0.8085 - val_loss: 0.4745 - val_acc: 0.7989
Epoch 232/500
15000/15000 [==============================] - 10s 683us/sample - loss: 0.4248 - acc: 0.8083 - val_loss: 0.4744 - val_acc: 0.7990
Epoch 233/500
15000/15000 [==============================] - 10s 680us/sample - loss: 0.4237 - acc: 0.8081 - val_loss: 0.4744 - val_acc: 0.7992
Epoch 234/500
15000/15000 [==============================] - 10s 681us/sample - loss: 0.4243 - acc: 0.8077 - val_loss: 0.4744 - val_acc: 0.7992
Epoch 235/500
15000/15000 [==============================] - 10s 690us/sample - loss: 0.4247 - acc: 0.8102 - val_loss: 0.4744 - val_acc:

Epoch 286/500
15000/15000 [==============================] - 10s 680us/sample - loss: 0.4248 - acc: 0.8071 - val_loss: 0.4739 - val_acc: 0.7989
Epoch 287/500
15000/15000 [==============================] - 10s 685us/sample - loss: 0.4212 - acc: 0.8109 - val_loss: 0.4739 - val_acc: 0.7989
Epoch 288/500
15000/15000 [==============================] - 10s 680us/sample - loss: 0.4227 - acc: 0.8083 - val_loss: 0.4739 - val_acc: 0.7991
Epoch 289/500
15000/15000 [==============================] - 10s 682us/sample - loss: 0.4198 - acc: 0.8100 - val_loss: 0.4738 - val_acc: 0.7996
Epoch 290/500
15000/15000 [==============================] - 10s 677us/sample - loss: 0.4228 - acc: 0.8072 - val_loss: 0.4738 - val_acc: 0.7995
Epoch 291/500
15000/15000 [==============================] - 10s 681us/sample - loss: 0.4208 - acc: 0.8084 - val_loss: 0.4738 - val_acc: 0.7996
Epoch 292/500
15000/15000 [==============================] - 10s 686us/sample - loss: 0.4254 - acc: 0.8071 - val_loss: 0.4738 - val_acc:

Epoch 343/500
15000/15000 [==============================] - 10s 669us/sample - loss: 0.4231 - acc: 0.8092 - val_loss: 0.4733 - val_acc: 0.7992
Epoch 344/500
15000/15000 [==============================] - 10s 676us/sample - loss: 0.4220 - acc: 0.8099 - val_loss: 0.4733 - val_acc: 0.7993
Epoch 345/500
15000/15000 [==============================] - 10s 677us/sample - loss: 0.4195 - acc: 0.8115 - val_loss: 0.4733 - val_acc: 0.7996
Epoch 346/500
15000/15000 [==============================] - 10s 675us/sample - loss: 0.4233 - acc: 0.8073 - val_loss: 0.4732 - val_acc: 0.7994
Epoch 347/500
15000/15000 [==============================] - 10s 680us/sample - loss: 0.4234 - acc: 0.8086 - val_loss: 0.4732 - val_acc: 0.7995
Epoch 348/500
15000/15000 [==============================] - 10s 687us/sample - loss: 0.4233 - acc: 0.8122 - val_loss: 0.4732 - val_acc: 0.7993
Epoch 349/500
15000/15000 [==============================] - 10s 681us/sample - loss: 0.4241 - acc: 0.8078 - val_loss: 0.4732 - val_acc:

Epoch 456/500
15000/15000 [==============================] - 10s 671us/sample - loss: 0.4184 - acc: 0.8127 - val_loss: 0.4721 - val_acc: 0.8004
Epoch 457/500
15000/15000 [==============================] - 10s 691us/sample - loss: 0.4215 - acc: 0.8150 - val_loss: 0.4721 - val_acc: 0.8007
Epoch 458/500
15000/15000 [==============================] - 10s 695us/sample - loss: 0.4212 - acc: 0.8113 - val_loss: 0.4721 - val_acc: 0.8002
Epoch 459/500
15000/15000 [==============================] - 10s 697us/sample - loss: 0.4207 - acc: 0.8107 - val_loss: 0.4721 - val_acc: 0.7997
Epoch 460/500
15000/15000 [==============================] - 11s 701us/sample - loss: 0.4199 - acc: 0.8125 - val_loss: 0.4721 - val_acc: 0.8000
Epoch 461/500
15000/15000 [==============================] - 11s 704us/sample - loss: 0.4200 - acc: 0.8123 - val_loss: 0.4721 - val_acc: 0.7994
Epoch 462/500
15000/15000 [==============================] - 11s 703us/sample - loss: 0.4219 - acc: 0.8104 - val_loss: 0.4721 - val_acc:

In [43]:
#Evaluating Model with Test DataSet
results = model.evaluate(test_data, test_labels)
#Printing Result of Trained Neural Network
print(results)

25000/25000 [==============================] - 5s 186us/sample - loss: 0.4734 - acc: 0.7984
[0.4734490588951111, 0.79844]
